In [66]:
import os
#os.chdir("/Users/postgres/Desktop/NLP/Text-Summarizer/src")
os.getcwd()

'/Users/postgres/Desktop/NLP/Text-Summarizer'

In [67]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_dir: Path
    unzip_dir: Path

In [68]:

from TextSummarizer.constants import *
from TextSummarizer.utils.common import *

In [69]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_dir=config.local_data_dir,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [70]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [71]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_dir):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_dir
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_dir))}")  

        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_dir, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [72]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-19 16:45:24,120] INFO: common: yaml file: config/config.yaml loaded successfully
[2023-09-19 16:45:24,121] INFO: common: yaml file: params.yaml loaded successfully
[2023-09-19 16:45:24,122] INFO: common: created directory at: artifacts
[2023-09-19 16:45:24,122] INFO: common: created directory at: artifacts/data_ingestion
[2023-09-19 16:45:25,009] INFO: 28827185: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E736:139F:3A368C:44A991:650A0863
Accept-Ranges: bytes
Date: Tue, 19 Sep 2023 20:45:24 GMT
Via: 1.1 varnish
X-Served-By: cache-lga21924-LGA
X-Cache: MISS
X-Ca